# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [49]:
import pandas as pd
import numpy as np
import glob
pd.options.mode.chained_assignment = None

DATA_FOLDER = './Data/' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

### GUINEA

First of all, we concatenate all the files in the guinea folder and we only take the 3 columns that interests us : the date, the description and the total for the whole country. We also parse the date with the right format.
Then we put the date as the index.

In [ ]:
path_guinea = DATA_FOLDER+"ebola/guinea_data/"
files_guinea = glob.glob(path_guinea+"*.csv")

df_guinea= pd.concat((pd.read_csv(f,usecols=["Date","Description","Totals"],parse_dates=["Date"]) for f in files_guinea),ignore_index=True)
df_guinea.index = df_guinea.Date
df_guinea = df_guinea.drop('Date',axis=1)
df_guinea

Now we filter the only 2 rows we need, here they are the new cases and the new deaths. We notice the first file uses a different description for the new deaths, so we replace it with the one that is used in the other files.

In [ ]:
filtered_guinea = df_guinea[(df_guinea.Description == "Total new cases registered so far") | (df_guinea.Description == "New deaths registered") | (df_guinea.Description == "New deaths registered today")]
filtered_guinea = filtered_guinea.replace("New deaths registered today","New deaths registered")

The pivot is just a convenient way of looking at it, we don't actually change the structure of the dataframe, because the current structure makes the following easier.

In [ ]:
filtered_guinea.pivot(columns="Description",values="Totals")

We now extract one serie for each (cases and deaths) and convert the values into integers.

In [ ]:
cases_guinea = filtered_guinea[filtered_guinea.Description == "Total new cases registered so far"].Totals.astype(str).astype(int)
deaths_guinea = filtered_guinea[filtered_guinea.Description == "New deaths registered"].Totals.astype(str).astype(int)
cases_guinea

This allows us to average by month quite easily :

In [ ]:
avg_cases_guinea = cases_guinea.resample('M').mean()
avg_deaths_guinea = deaths_guinea.resample('M').mean()
avg_cases_guinea

And we can finally present the data in a nice, concise way.

In [ ]:
avg_guinea = pd.DataFrame({"avg new cases":avg_cases_guinea.values,"avg new deaths":avg_deaths_guinea.values},index=avg_cases_guinea.index.strftime('%B'))
avg_guinea

### LIBERIA

We proceed the same way, only with different names for columns and rows.

In [ ]:
path_liberia = DATA_FOLDER+"ebola/liberia_data/"
files_liberia = glob.glob(path_liberia+"*.csv")

df_liberia= pd.concat((pd.read_csv(f,usecols=["Date","Variable","National"],parse_dates=["Date"]) for f in files_liberia),ignore_index=True)
df_liberia.columns=["Date","Description","Totals"] #for clarity we use the same column names as before
df_liberia.index = df_liberia.Date
df_liberia = df_liberia.drop('Date',axis=1)

In [ ]:
filtered_liberia = df_liberia[(df_liberia.Description.str.contains("New Case|New case")) | (df_liberia.Description == "Newly reported deaths")]

Here, we have to considerate all the different types of new cases and sum them by day.

In [ ]:
separate_cases = filtered_liberia[filtered_liberia.Description != "Newly reported deaths"]
cases_liberia = separate_cases.groupby(separate_cases.index).sum().Totals
deaths_liberia = filtered_liberia[filtered_liberia.Description == "Newly reported deaths"].Totals

In [ ]:
avg_cases_liberia = cases_liberia.resample('M').mean()
avg_deaths_liberia = deaths_liberia.resample('M').mean()

In [ ]:
avg_liberia = pd.DataFrame({"avg new cases":avg_cases_liberia.values,"avg new deaths":avg_deaths_liberia.values},index=avg_cases_liberia.index.strftime('%B'))
avg_liberia

### SIERRA LEONE

In [ ]:
path_sierra = DATA_FOLDER+"ebola/sl_data/"
files_sierra = glob.glob(path_sierra+"*.csv")

df_sierra= pd.concat((pd.read_csv(f,usecols=["date","variable","National"],parse_dates=["date"]) for f in files_sierra),ignore_index=True)
df_sierra.columns=["Date","Description","Totals"] #for clarity we use the same column names as before
df_sierra.index = df_sierra.Date
df_sierra = df_sierra.drop('Date',axis=1)

In [ ]:
filtered_sierra = df_sierra[(df_sierra.Description.str.contains("new_suspected|new_probable|new_confirmed")) | (df_sierra.Description == "etc_new_deaths")]

As for Liberia, we have to sum all the different types of new cases (suspected, probable, confirmed).

In [ ]:
separate_cases = filtered_sierra[filtered_sierra.Description != "etc_new_deaths"].Totals.astype(str).astype(float)
cases_sierra = separate_cases.groupby(separate_cases.index).sum()
deaths_sierra = filtered_sierra[filtered_sierra.Description == "etc_new_deaths"].Totals.astype(str).astype(float)

In [ ]:
avg_cases_sierra = cases_sierra.resample('M').mean()
avg_deaths_sierra = deaths_sierra.resample('M').mean()

In [ ]:
avg_sierra = pd.DataFrame({"avg new cases":avg_cases_sierra.values,"avg new deaths":avg_deaths_sierra.values},index=avg_cases_sierra.index.strftime('%B'))
avg_sierra

### Concatenating all the results in one dataframe

We can note that one value, the average number of new cases in Liberia in december, is strongly out of the domain formed by the other values. This is due to the data (after research, peak might be due to delayed reporting of older cases) and not the calculations.

In [ ]:
final_result = pd.concat([avg_guinea,avg_liberia,avg_sierra],axis=1,keys=["Guinea","Liberia","Sierra"]).reindex(avg_liberia.index)
final_result.fillna("unknown",inplace=True)
final_result

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
path=DATA_FOLDER+"microbiome/"
MID_files = glob.glob(path+"MID*.xls")

After we extract all the 9 spreadsheets from the folder, the metadata file is imported into a DataFrame and we can see that the file gives two informations (Group and Sample) for each MID file.

In [ ]:
df_metadata=pd.read_excel('Data/microbiome/metadata.xls',index_col='BARCODE')
df_metadata

Each spreadsheet is composed of two columns : the name of the bacteria and the number found in the patient.
We define the name of the bacteria as index for each file and use the name of the file (MID1, MID2, ...) as column name in order to anticipate the merge with the metadata file. 
The 9 spreadsheets are imported into a single DataFrame through a column-wise concatenation (axis=1) with respect to the files indices.  
The goal is to add the information contained in the metadata file as columns in our DataFrame that's why we need to transpose our DataFrame in order to have the MID files in rows and the bacteria counts + metadata informations in columns.

In [ ]:
df_microbiome=pd.concat((pd.read_excel(f,index_col=0,header=None,names=[f[f.find('MID'):f.find('MID')+4]]) for f in MID_files),axis=1)
df_microbiome=df_microbiome.transpose()
df_microbiome.index.names=['BARCODE']
df_microbiome

Then we can add to our DataFrame the information contained in the metadata file, doing an inner join which is the default method for a merge in *Pandas*. The only column which is the same between the two DataFrame is the index one (BARCODE) and this needs to be specify in the merge function with the left_index and right_index arguments.

In [ ]:
df_merged=pd.merge(df_microbiome, df_metadata, left_index=True, right_index=True)
df_merged

Finally, we can check the required constraints which are :
- the Nan values needs to be replaced by 'unknown'
- the index needs to be unique, which is the case because we use the name of each spreadsheet as indices

In [ ]:
df_merged.fillna('unknown',inplace=True)
df_merged

In [ ]:
df_merged.index.is_unique

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [50]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

Name,Labels,Units,Levels,Storage,NAs
pclass,,,3,integer,0
survived,Survived,,,double,0
name,Name,,,character,0
sex,,,2,integer,0
age,Age,Year,,double,263
sibsp,Number of Siblings/Spouses Aboard,,,double,0
parch,Number of Parents/Children Aboard,,,double,0
ticket,Ticket Number,,,character,0
fare,Passenger Fare,British Pound (\243),,double,1
cabin,,,187,integer,0


For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [51]:
# import titanic.xls

tf = pd.read_excel('./Data/titanic.xls')
tf = tf.rename(columns={'home.dest':'homedest'})

In [52]:
tf

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,homedest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
5,1,1,"Anderson, Mr. Harry",male,48.0000,0,0,19952,26.5500,E12,S,3,NaN,"New York, NY"
6,1,1,"Andrews, Miss. Kornelia Theodosia",female,63.0000,1,0,13502,77.9583,D7,S,10,NaN,"Hudson, NY"
7,1,0,"Andrews, Mr. Thomas Jr",male,39.0000,0,0,112050,0.0000,A36,S,NaN,NaN,"Belfast, NI"
8,1,1,"Appleton, Mrs. Edward Dale (Charlotte Lamson)",female,53.0000,2,0,11769,51.4792,C101,S,D,NaN,"Bayside, Queens, NY"
9,1,0,"Artagaveytia, Mr. Ramon",male,71.0000,0,0,PC 17609,49.5042,NaN,C,NaN,22.0,"Montevideo, Uruguay"


In [53]:
pd.DataFrame({'Min':pd.DataFrame.min(tf),'Max':pd.DataFrame.max(tf) })

,Max,Min
pclass,3.0000,1.0000
survived,1.0000,0.0000
age,80.0000,0.1667
sibsp,8.0000,0.0000
parch,9.0000,0.0000
fare,512.3292,0.0000
body,328.0000,1.0000


In [54]:
tf.sex.unique()

array(['female', 'male'], dtype=object)

In [55]:
tf.embarked.unique()

array(['S', 'C', nan, 'Q'], dtype=object)

### 1. Type and value range of each attribute
- Pclass : integer value in range 1-3
- Survived : integer value in range 0-1
- Name : string
- Sex : string 'female' or 'male'
- Age : double value in range 0.1667 - 80
- Sibsp : double value in range 0-8
- Parch : double value in range 0-9
- Ticket : string 
- Fare : double value in range 0-512.3292
- Cabin : string
- Embarked : character that can take value S, C or Q
- Boat : string
- Body : double value in range 1-328
- Home destination : String 

We could say that the attributes 'pclass', 'sex', 'survived', and 'embarked' can be categorical. We then cast them as category.


In [56]:
# transform the concerned attributes into categories
tf['pclass'] = tf.pclass.astype('category')
tf['sex'] = tf.sex.astype('category')
tf['survived'] = tf.survived.astype('category')
tf['embarked'] = tf.embarked.astype('category')

### 2. Histograms

In [57]:
import matplotlib.pyplot as plt
import seaborn as sns

# histogram function
def hist(t, data, labs, rot):
    fig, ax = plt.subplots()
    ax.set_title(t, fontsize=15, fontweight='bold')
    sns.barplot(labs, data, ax=ax)
    locs, labels = plt.xticks()
    plt.setp(labels, rotation=rot)
    plt.show()

- Travel class

In [58]:
hist('Travel Class', tf.pclass.value_counts(sort=False), tf.pclass.value_counts(sort=False).keys(), 0)

AttributeError: 'SeriesGroupBy' object has no attribute 'indices'

- Embarkation port

In [ ]:
hist('Embarkation port', tf.embarked.value_counts(sort=False), tf.embarked.value_counts(sort=False).keys(), 0)

- Sex

In [ ]:
hist('Gender', tf.sex.value_counts(sort=False), tf.sex.value_counts(sort=False).keys(), 0)

- Age (Discrete decades interval)

In [ ]:
# divide data into age intervals
decade_int = pd.cut(tf.age, [0,9,19,29,39,49,59,69,79, 89], labels=['<10','10\'s','20\'s','30\'s', '40\'s', '50\'s', '60\'s','70\'s', '80\'s'])

hist("Age", decade_int.value_counts(sort=False), decade_int.value_counts(sort=False).keys(), 0)

### 3. Passengers on each cabin floor

In [ ]:
#colors = ['gold', 'yellowgreen', 'lightcoral', 'lightskyblue']
# pie chart function
def piechart(t, counts, labels):
    #plt.pie(counts, labels=labels, autopct='%1.1f%%', shadow=False, startangle=140)
    patches, texts = plt.pie(counts, startangle=90)
    plt.legend(patches, labels, loc="best")
    plt.axis('equal')
    plt.title(t)
    plt.tight_layout()
    plt.show()

In [ ]:
# distinguish the different floors
floors = tf['cabin'].dropna().astype(str).str[0]
floors = floors.astype('category')

piechart("Passengers repartition on floors", floors.value_counts(), floors.cat.categories)


### 4. Survivors for each travel class

In [ ]:
# group the datas by travel class
pclass_grouped = tf.groupby(tf.pclass)

In [ ]:
for pc, group in pclass_grouped:
    piechart("Survivers in class " + str(pc), group.survived.value_counts(sort=False), ["Deads", "Survivors"])

### 5. Proportion of survivors by travel class and sex

In [ ]:
def percent(x):
    return 100*x[1]/(x[0]+x[1])

# group the datas by travel class and sex
trav_sex_grouped = tf.groupby(['pclass', 'sex'])
values_hist = list()
labs = list()

for p, group in trav_sex_grouped:
    res = group.survived.value_counts(sort=False)
    values_hist.append(percent((group.survived.value_counts(sort=False).values)))
    labs.append(str(p[1]) + " in class " + str(p[0]) + "")


In [ ]:
hist("Prop", values_hist, labs, 67)

### 6. Survival proportion by age category, travel class and sex

In [ ]:
# separate population in 2 age equal intervals
tf_age = pd.qcut(tf.age, 2)

# group the datas by travel class, sex and our age interval
tf_grouped = tf.groupby(['pclass', 'sex', tf_age])

In [ ]:
indx = list()
data_res = list()
for p, group in tf_grouped :
    indx.append(str(p[1]+" in class "+str(p[0]) + ", age in " + str(p[2])))
    data_res.append(percent((group.survived.value_counts(sort=False)).values))

In [ ]:
pd.DataFrame(data_res, index=indx, columns=['Percentage']).transpose()